# Structured Output from LLMs (LlamaIndex | LlaMa-3 | Groq API)

### Table of Contents

1. **Load LLM**
2. **Define Output Schemas (Pydantic Classes)**
3. **Generate Pydantic | JSON Output**


In [1]:
import dotenv
import os

dotenv.load_dotenv(dotenv.find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]

## 1. Load LLM

* Login to **https://console.groq.com** and create API Key.

### Groq Models

ID|	REQUESTS PER MINUTE|	REQUESTS PER DAY|	TOKENS PER MINUTE
-|-|-|-
llama3-70b-8192	|30	|14,400	|6,000
llama3-8b-8192	|30	|14,400	|30,000
gemma-7b-it	|30	|14,400	|15,000
gemma2-9b-it |30	|14,400	|15,000
mixtral-8x7b-32768	|30	|14,400	|5,000
llama3-groq-8b-8192-tool-use-preview|30	|14,400	|15,000
llama3-groq-70b-8192-tool-use-preview|30	|14,400	|15,000


#### LlamaIndex Groq Package

* **pip install llama-index-llms-groq**

In [2]:
from llama_index.llms.groq import Groq

llama3 = Groq(model="llama3-8b-8192", api_key=groq_api_key, temperature=0.0)

llama3

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f9b186ab4c0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f9ad4c52b00>, completion_to_prompt=<function default_completion_to_prompt at 0x7f9ad4cc7490>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama3-8b-8192', temperature=0.0, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [3]:
llama3.complete("Hello, How are you?")

CompletionResponse(text="Hello! I'm just an AI, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How can I assist you today?", additional_kwargs={}, raw={'id': 'chatcmpl-b8dac03d-3395-474f-b1a9-fc68b7e200c9', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just an AI, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How can I assist you today?", role='assistant', function_call=None, tool_calls=None))], 'created': 1721188753, 'model': 'llama3-8b-8192', 'object': 'chat.completion', 'system_fingerprint': 'fp_af05557ca2', 'usage': CompletionUsage(completion_tokens=46, prompt_tokens=16, total_tokens=62, prompt_time=0.003540944, completion_time=0.035589898, total_time=0.039130842), 'x_groq': {'id': 'req_01j2zdg83kebgbha4x9cgkbx40'}}, logprobs=

## 2. Define Schemas

In [4]:
from typing import Optional, List
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Class Representing Individual Person."""

    name: str = Field(description="Name of the person.")
    age: int = Field(description="Age of Person.")
    height: Optional[str] = Field(description="Height of Person")

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]

## 3. Generate Structured Output (Pydantic / JSON Objects)

In [5]:
from llama_index.core.program import LLMTextCompletionProgram, FunctionCallingProgram
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
You are an expert data parser. Parse data from user query.

If you don't know any field then set it to None.

{query}
"""

pydantic_data_parser = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Person),
    #output_cls=Person,
    prompt_template_str=prompt_template_str,
    llm=llama3
)

In [6]:
response = pydantic_data_parser(query="Anna is 20 years old and five foot five inch.")

response

Person(name='Anna', age=20, height='five foot five inch')

In [7]:
response.dict()

{'name': 'Anna', 'age': 20, 'height': 'five foot five inch'}

In [8]:
response.json()

'{"name": "Anna", "age": 20, "height": "five foot five inch"}'

In [9]:
response = pydantic_data_parser(query="Sam is 25 years old and 5 foot 10 inch.")

response

Person(name='Sam', age=25, height='5 foot 10 inch')

In [10]:
response.dict()

{'name': 'Sam', 'age': 25, 'height': '5 foot 10 inch'}

In [11]:
pydantic_data_parser = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(People),
    #output_cls=People,
    prompt_template_str=prompt_template_str,
    llm=llama3
)

response = pydantic_data_parser(query="Anna is 20 years old and five foot five inch.")

response

People(people=[Person(name='Anna', age=20, height='five foot five inch')])

In [12]:
response.dict()

{'people': [{'name': 'Anna', 'age': 20, 'height': 'five foot five inch'}]}

In [13]:
response = pydantic_data_parser(query="Anna is 20 years old and five foot five inch. Sam is 25 years old and 5 foot 10 inch.")

response

People(people=[Person(name='Anna', age=20, height='five foot five inch'), Person(name='Sam', age=25, height='5 foot 10 inch')])

In [14]:
response.dict()

{'people': [{'name': 'Anna', 'age': 20, 'height': 'five foot five inch'},
  {'name': 'Sam', 'age': 25, 'height': '5 foot 10 inch'}]}

In [15]:
query = """
Anna is 20 years old and five foot five inch. She lives in UK. Currently, she is working as marketing manager.

Sam is 25 years old and 5 foot 10 inch. He lives in US. Currently, he is working as product manager at JP Morgan.

Donna is 35 years old and 5 foot 8 inch. She lives in Singapore. Currently, she is working as developer advocate.

Jack is 45 years old and 6 foot 2 inch. He lives in Germany. Currently, He is CEO at Skype.

Elon is 55 years old and 6 foot tall. He lives in US. Currently, He is CEO of Tesla.
"""

response = pydantic_data_parser(query=query)

response

People(people=[Person(name='Anna', age=20, height='5 foot 5 inch'), Person(name='Sam', age=25, height='5 foot 10 inch'), Person(name='Donna', age=35, height='5 foot 8 inch'), Person(name='Jack', age=45, height='6 foot 2 inch'), Person(name='Elon', age=55, height='6 foot')])

In [16]:
response.dict()

{'people': [{'name': 'Anna', 'age': 20, 'height': '5 foot 5 inch'},
  {'name': 'Sam', 'age': 25, 'height': '5 foot 10 inch'},
  {'name': 'Donna', 'age': 35, 'height': '5 foot 8 inch'},
  {'name': 'Jack', 'age': 45, 'height': '6 foot 2 inch'},
  {'name': 'Elon', 'age': 55, 'height': '6 foot'}]}

## Summary

In this video, I explained how to generate **structured output (Pydantic | JSON)** using **Open Source LLMs (LlaMa-3)**. For coding, we used **LlamaIndex** framework.